In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os 
import pandas as pd

In [2]:
Page_Number_Start = 1
Page_Number_End = 3


In [3]:
def Page_Loop():
    df = pd.DataFrame({"Case_Info":[], "Neutral_CItation_No":[], "Date_of_Judgement":[], "Title":[], "Path":[]})
    for Page in range(Page_Number_Start, Page_Number_End+1, 1):
        df = Extract_Page_Info(Page, df)    
    return df

In [4]:
def Extract_Page_Info(Page, df):
    Download_id = []
    Doc_Info = []
    Case_No = []
    Neutral_Citation_No = []
    Date_of_Judgement = []
    Title = []
    Archive_URL = "https://legalref.judiciary.hk/lrs/common/ju/tjpv.jsp?page=" + str(Page) + "&CL=&I_SORT=s_ud&I_SORT_ORDER=1"
    #print(Archive_URL)
    Archive_Page = requests.get(Archive_URL)
    Archive_Page_Soup = BeautifulSoup(Archive_Page.content,'html.parser')
    #print(Archive_Page_Soup.prettify())
    Download_id_tag = Archive_Page_Soup.find_all('a', attrs={"class": "smalltxt"})
    Download_id_pattern = re.compile("=(?P<caseID>\d+)")
    for i in range(len(Download_id_tag)):
        #print(i)
        #print(Download_id_tag[i])
        Download = Download_id_tag[i]['href']
        Download_ID = re.search(Download_id_pattern, Download)
        Download_id.append(Download_ID.group('caseID'))
        Title.append(Download_id_tag[i].text)
    #print(Download_id)
    #print(Title)
    Case_number = Archive_Page_Soup.find_all('td', attrs={"align":"left", "class":"smalltxt", "nowrap":"", "style":"border-right:1px solid #A35910;", "valign":"top"})
    for i in range(len(Case_number)):
        Case_No.append(Case_number[i].text.replace(" ",""))
    #print(Case_No)
    Others = Archive_Page_Soup.find_all('td', attrs={"valign":"top", "align":"center", "class":"smalltxt", "style":"border-right:1px solid #A35910;"})
    for i in range(len(Others)):
        if i % 2 == 0:
            Neutral_Citation_No.append(Others[i].text)
        else:
            Date_of_Judgement.append(Others[i].text)
    #print(Neutral_Citation_No)
    #print(Date_of_Judgement)
    for i in range(len(Download_id)):
        df = Extract_Document(Download_id[i], Case_No[i], Neutral_Citation_No[i], Date_of_Judgement[i], Title[i], df)
    return df

In [5]:
def Extract_Document(id, Case_Info, Neutral_CItation_No, Date_of_Judgement, Title, df):
    #print(str(id)
    DocURL = "https://legalref.judiciary.hk/lrs/common/ju/ju_body.jsp?DIS=" + str(id) + "&AH=&QS=&FN=&currpage=T#"
    #print(DocURL)
    Doc_Page = requests.get(DocURL)
    Doc_Page_Soup = BeautifulSoup(Doc_Page.content, 'html.parser')
    DocContent = str(Doc_Page_Soup.find('body'))
    Doc_Title = Doc_Page_Soup.title.text.replace("/", "-")
    #print(Doc_Title)
    Path = r"./Document/" + str(Title) + ".html"
    DF_Dict = {'Case_Info': str(Case_Info), 'Neutral_CItation_No': str(Neutral_CItation_No), 'Date_of_Judgement': str(Date_of_Judgement), 'Title': str(Title), 'Path': str(Path)}
    #print(DF_Dict)
    #print(DF_Info)
    Create_Dir()
    Save_As_HTML(Doc_Title,DocContent)
    df = DataFrameRecord(DF_Dict, df)
    return df

In [6]:
def DataFrameRecord(Record, df):
    return df.append(Record, ignore_index=True)

In [7]:
def Save_As_HTML(Title,Content):
    path = r"./Document/" + str(Title) + ".html"
    Text_File = open(path, "w")
    Text_File.write(Content)
    Text_File.close 

In [8]:
def Create_Dir():
    if not (os.path.exists("./Document")):
        os.mkdir("./Document")

In [9]:
df = Page_Loop()

In [10]:
df

,Case_Info,Neutral_CItation_No,Date_of_Judgement,Title,Path
0,HCMA492/2019,[2020] HKCFI 2425,17/09/2020,HKSAR v. NG MAN YUEN AVERY AND OTHERS,./Document/HKSAR v. NG MAN YUEN AVERY AND OTHE...
1,HCMA21/2020,[2020] HKCFI 2436,21/09/2020,HKSAR v. YAU KAM LUNG,./Document/HKSAR v. YAU KAM LUNG.html
2,HCMA580/2019,[2020] HKCFI 2235,04/09/2020,HKSAR v. LI TSZ TUNG YVONNE,./Document/HKSAR v. LI TSZ TUNG YVONNE.html
3,CACC94/2019,[2020] HKCA 767,24/09/2020,HKSAR v. CHOI CHI TAK,./Document/HKSAR v. CHOI CHI TAK.html
4,CAAR1/2020,[2020] HKCA 788,22/09/2020,SECRETARY FOR JUSTICE v. SWS,./Document/SECRETARY FOR JUSTICE v. SWS.html
5,HCMA101/2020,[2020] HKCFI 1808,06/08/2020,HKSAR v. YUNG WAI SHING AND ANOTHER,./Document/HKSAR v. YUNG WAI SHING AND ANOTHER...
6,CAAR7/2020,[2020] HKCA 829,28/10/2020,SECRETARY FOR JUSTICE v. S H Y,./Document/SECRETARY FOR JUSTICE v. S H Y.html
7,HCMA20/2020,[2020] HKCFI 1783,07/08/2020,HKSAR v. FU GUANGMIAO,./Document/HKSAR v. FU GUANGMIAO.html
8,HCMA619/2018,[2020] HKCFI 1966,21/08/2020,HKSAR v. CHONG CHI KEUNG,./Document/HKSAR v. CHONG CHI KEUNG.html
9,CACC5/2019,[2020] HKCA 295,14/05/2020,HKSAR v. KWAN WAN KI,./Document/HKSAR v. KWAN WAN KI.html
